In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

def fetch_medicamento(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        return None
    
def trocar_valor(valor, equivalencias):
    if valor in equivalencias:
        return equivalencias[valor]
    else:
        return valor

equivalencias = {
    "Dor e Febre": "Dor E Febre",
    "Digestão": "Digestivo",
    "Alergias": "Alergias E Infecções",
    "Pressão Alta": "Hipertensão",
    "Emagrecimento": "Emagrecedores",
    "Sistema Imunológico": "Gripe E Resfriado",
    "Tratamento Capilar": "Similar",
    "Antitabagismo": "Similar",
    "Saúde Feminina": "Anticoncepcionais",
    # O restante dos medicamentos será agrupado como "Mais Medicamentos"
    "Sistema Respiratório": "Mais Medicamentos",
    "Sistema Circulatório": "Mais Medicamentos",
    "Pele e Mucosa": "Mais Medicamentos",
    "Olhos": "Mais Medicamentos",
    "Hormônios": "Mais Medicamentos",
    "Para os Ossos": "Mais Medicamentos",
    "Sistema Nervoso": "Mais Medicamentos",
    "Saúde Masculina": "Mais Medicamentos",
    "Oncológicos e Alto Custo": "Mais Medicamentos",
    "Vermes E Parasitas": "Mais Medicamentos",
    "Ouvido": "Mais Medicamentos",
    "Herpes": "Mais Medicamentos"
}

def process_medicamento(medicamento):
    try:
        nome = medicamento.find("h1", class_="name").text.strip()
    except:
        nome = None
    try: 
        preco = float(medicamento.find("p", class_="unit-price").text.strip().replace('R$ ', '').replace(',', '.'))
    except: 
        preco = 0
    try: 
        desconto = medicamento.find("span", class_="descont").text.strip().replace('%', '').replace('-', '')
    except: 
        desconto = 0
    try:
        preco_desconto = medicamento.find("meta", itemprop='price').get('content')
    except:
        preco_desconto = 0
    try:
        ean = medicamento.find("meta", itemprop='gtin13').get('content')
    except:
        ean = None
    try:
        marca = medicamento.find("meta", itemprop='brand').get('content')
    except:
        marca = None
    try:
        setas = medicamento.find_all("span", itemprop='name')

        for dado in setas:
            if dado.text != "Início" and dado.text != "Medicamentos":
                categoria = dado.text
                categoria = trocar_valor(categoria,equivalencias)
                break
    except:
        categoria = None

    return nome, preco, preco_desconto, desconto, ean, marca, categoria



def main():
    URLS=[]
    urls_primaria = []
    for i in range(1, 156):
        # URL da página de medicamentos
        urls_primaria.append("https://www.drogariaveracruz.com.br/medicamentos/?p="+str(i))

    with ThreadPoolExecutor(max_workers=10) as executor:
        responses_primaria = list(executor.map(fetch_medicamento, urls_primaria))
        
        for response in responses_primaria:
            # Criar objeto BeautifulSoup
            soup = BeautifulSoup(response, "html.parser")
            # Encontrar os containers dos medicamentos
            medicamentos = soup.find_all("div", class_="li")
            # Extrair dados para cada medicamento

            for medicamento in medicamentos:
                add = medicamento.find("a", class_='item-image').get('href') 
                URL = "https://www.drogariaveracruz.com.br" + add
                URLS.append(URL)
                
    print(len(URLS))
    print(URLS[0])

    NOME = []
    PRECO=[]
    PRECO_DESCONTO = []
    DESCONTO = []
    EAN=[]
    MARCA=[]
    CATEGORIA=[]

    with ThreadPoolExecutor(max_workers=12) as executor:
        responses = list(executor.map(fetch_medicamento, URLS))
        
        for response in responses:
            if response is not None:
                soup = BeautifulSoup(response, "html.parser")
                medicamentos = soup.find_all("div", class_="content")
                for medicamento in medicamentos:
                    nome, preco, preco_desconto, desconto, ean, marca, categoria = process_medicamento(medicamento)
                    NOME.append(nome)
                    PRECO.append(preco)
                    PRECO_DESCONTO.append(preco_desconto)
                    DESCONTO.append(desconto)
                    EAN.append(ean)
                    MARCA.append(marca)
                    CATEGORIA.append(categoria)

    df = pd.DataFrame()
    df['Nome'] = NOME
    df['Preço sem desconto'] = PRECO
    df['Preço com desconto'] = PRECO_DESCONTO
    df['Desconto (porcentagem)'] = DESCONTO 
    df['Código EAN'] = EAN
    df['Marca'] = MARCA
    df['Categoria'] = CATEGORIA
    print(df)

    return df



In [2]:
df = main()

3098
https://www.drogariaveracruz.com.br/suplemento-alimentar-probid-30-capsulas/p
                                                   Nome  Preço sem desconto  \
0               Suplemento Alimentar Probid 30 Cápsulas               78.90   
1     Suplemento Vitamínico Nestlé Materna 30 compri...               71.90   
2                            Lactosil Flora 30 cápsulas               60.90   
3     Solução Multiuso Renu Fresh 355ml + 120ml + Es...               44.99   
4     Suplemento Alimentar Calcitran Triflex 30 comp...               61.90   
...                                                 ...                 ...   
3093  Espinheira Santa Gastriless 380mg, Bionatus 45...                0.00   
3094          Esperson 2,5mg/g Pomada Dermatológica 20g                0.00   
3095  PICATO - 150mcg/g, 3 bisnagas com 0,47g de gel...                0.00   
3096  LOVELLE - 0,25mg + 0,05mg, 21 comprimidos reve...                0.00   
3097          Piascledine 300 100mg + 200mg 30 c

In [3]:
# Carregar o arquivo CSV como um DataFrame
df.to_excel('Drogaria_VeraCruz_Excel.xlsx')

# Exibir as primeiras linhas do DataFrame para verificar se foi carregado corretamente
df.head()

,Nome,Preço sem desconto,Preço com desconto,Desconto (porcentagem),Código EAN,Marca,Categoria
0,Suplemento Alimentar Probid 30 Cápsulas,78.90,68.64,13,7896637032209,Apsen,Digestivo
1,Suplemento Vitamínico Nestlé Materna 30 compri...,71.90,66.15,8,7891045031257,Nestlé,Anticoncepcionais
2,Lactosil Flora 30 cápsulas,60.90,53.59,12,7896637032247,Apsen,Digestivo
3,Solução Multiuso Renu Fresh 355ml + 120ml + Es...,44.99,37.34,17,7896046381004,Bausch & Lomb,Mais Medicamentos
4,Suplemento Alimentar Calcitran Triflex 30 comp...,61.90,56.76,8,7898040329716,Divcom Pharma,Mais Medicamentos
